In [1]:
import pickle
import json
import nbformat
import os
from PIL import Image
from PIL.PngImagePlugin import PngInfo
from pyperiscope import Scope

class Pilot():
    def __init__(self, workbook, step_timeout=1):
        self.current_notebook = workbook
        self.steps = []
        self.current_step = -1
        self.steps_total = -1
        self.current_doc = ""
        self.last_run_code = ""
        self.last_error = ""
        self.scope_cell_id = -1
        self.step_timeout = step_timeout
        
        self.get_steps()

    # load the steps form automation workbook for fruthert usage
    def get_steps(self):
        with open(self.current_notebook, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        # Find all cells with the marker comment
        marker_cells = []
        for i, cell in enumerate(notebook['cells']):
            if cell['cell_type'] == 'code' and cell['source']:
                if cell['source'][0].find("# comment: Automated step generated with pyPeriscope")>-1:
                    marker_cells.append(i)
        
        if not marker_cells:
            self.steps_total = -1
            print("No marker cells found, are you sure this is the correct jupyter lab notebook?")
            return
        
        # Create step ranges
        self.steps = []
        # First step: from start to first marker
        self.steps.append((0, marker_cells[0]-1))
        
        # Middle steps: between markers
        for i in range(len(marker_cells)-1):
            self.steps.append((marker_cells[i]-1, marker_cells[i+1]-1))
        
        # Last step: from last marker to end
        if marker_cells:
            self.steps.append((marker_cells[-1]-1, len(notebook['cells'])))
    
        # Make the 1st two into one as there are some imports in front
        self.steps = self.steps[1:]
        self.steps[0] = (0,self.steps[0][1])
        
        self.steps_total = len(self.steps)

    # read one cell worth of data
    def get_cell(self, cell_number):
        with open(self.current_notebook, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        code_cells = [cell for cell in notebook['cells']]
        
        if cell_number < 0 or cell_number >= len(code_cells):
            raise ValueError(f"Cell number must be between 0 and {len(code_cells) - 1}")
        
        return code_cells[cell_number]

    # simplified view cell (remove hs object defintions and outputs, use for debug)
    def view_cell(self, cell_number):
        code = self.get_cell(cell_number)
        if 'source' in code:
            if code['source'][0].find("# comment: Automated step") > -1:
                code['source'][1] = 'payload = \'*** removed for view ***\''
        if 'outputs' in code:
            if len(code['outputs']) > 0:
                code['outputs'] = ['*** removed for view ***']
        
        return(code)
    
    # run the steps in global namespace so it would work like running the worksheet on the worksheet
    def run_code(self, code_str):
        self.last_run_code = code_str
        current_globals = globals()  # Get fresh globals each time
        exec(code_str, current_globals)

    # save screenshot with step and data
    def save_screenshot_with_data(self, screenshot, step, step_no):
        data_dict = {}
        # save the step dict
        data_dict['step'] = step.save_dict()
        data_dict['last_found'] = step.found_locations
        # save current step
        data_dict['current_step'] = self.current_step
        # save last error
        data_dict['last_error'] = self.last_error
        # last step scope cell id
        data_dict['scope_cell_id'] = self.scope_cell_id
        # docstring into saved png
        data_dict['current_doc'] = self.current_doc
        
        metadata = PngInfo()
        # Convert dict to bytes using pickle
        serialized_data = pickle.dumps(data_dict)
        # Store in PNG metadata with custom key
        metadata.add_text("custom_data", serialized_data.hex())
        screenshot.save("_"+self.current_notebook[:-6]+"-"+str(step_no)+".png", "PNG", pnginfo=metadata)

    # replace the scope object in the current step
    def replace_scope(self, scope_in):
        with open(self.current_notebook, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        
        # Find and edit the cell with matching ID
        for cell in nb.cells:
            if cell.get('id') == self.scope_cell_id:
                cell.source = scope_in.get_string()
                break
        
        with open(self.current_notebook, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
    
    # run single cell from notebook
    def run_cell(self, cell_number):
        self.last_error = None
        # get cell content
        current_cell = self.get_cell(cell_number)
        # if docstring cell update the variable
        if not current_cell['cell_type'] == 'code':
                if current_cell['cell_type'] == 'markdown':
                    content = "".join(current_cell['source'])
                    self.current_doc = self.current_doc + content
                else:
                    print("cell in not excecutable in 'code' mode, current mode: "+str(current_cell['cell_type']))
                return
        # join code into single string from line list
        current_code = "".join(current_cell['source'])
        # if the cell contains scope deffintion
        if current_code.find("# comment: Automated step") > -1:
            self.scope_cell_id = current_cell['id']
        # remove render preview
        code = current_code.replace("step.render_preview()\n","")
        # after all run the code
        try:
            self.run_code(code)
        except Exception as e:
            self.last_error = e
        # stop if there was an error
        if self.last_error:
            return

    # run one step end to end
    def run_step(self, step_no):
        print("Staring step: " + str(step_no))
        self.current_doc = ""
        current_screenshot = pyautogui.screenshot()
        for active_cell in range(self.steps[step_no][0], self.steps[step_no][1]):
            self.current_step = step_no
            self.run_cell(active_cell)
            if self.last_error:
                print(self.last_error)
                break
        self.save_screenshot_with_data(current_screenshot, step, step_no)

    # select and run a workbook
    def run_workbook(self, firstStep = -1):
        # run all the steps in the workbook
        for step in range(0, len(self.steps)):
            # if the workflow broke donw the continue from the last step
            if firstStep > 0:
                if firstStep == step:
                    firstStep = -1
                else:
                    continue
            # run the indivdual step
            self.run_step(step)  
                    
            # if error then stop, error is printed in the step process
            if self.last_error:
                return

            # cleanup the previous step and use timeout
            self.del_previous(step)    
            time.sleep(self.step_timeout)
        # delete the last log image if all was sucessfull
        self.del_previous(len(self.steps)) 

    # delete previous workbook
    def del_previous(self, current):
        if current > 0:
                previous_file = "_" + self.current_notebook[:-6] + "-" + str(current - 1) + ".png"
                if os.path.exists(previous_file):
                    os.remove(previous_file)

In [2]:
import os
import threading
import time
from PIL import Image, ImageFile

# Enable truncated image loading
ImageFile.LOAD_TRUNCATED_IMAGES = True

class FileWatcher:
    """Robust file watcher with write completion detection"""
    def __init__(self, binoculars, base_path):
        self.binoculars = binoculars
        self.base_path = base_path
        self.current_index = 0
        self.running = False
        self.thread = None
        self.lock = threading.Lock()

    def get_current_path(self):
        return f"{self.base_path}-{self.current_index}.png"

    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self.monitor_sequence)
        self.thread.daemon = True
        self.thread.start()

    def stop(self):
        self.running = False
        if self.thread:
            self.thread.join()

    def monitor_sequence(self):
        while self.running:
            current_path = self.get_current_path()
            if self._wait_for_file(current_path):
                try:
                    with self.lock:
                        self.binoculars._update_from_screenshot(current_path)
                        self.current_index += 1
                        print(f"Waiting for {self.get_current_path()}")
                except Exception as e:
                    print(f"Error processing {current_path}: {str(e)}")
                    self.current_index += 1  # Prevent blocking
            else:
                time.sleep(0.1)

    def _wait_for_file(self, path, timeout=5):
        """Wait for file to stabilize and become readable"""
        start_time = time.time()
        last_size = -1
        stable_count = 0

        while time.time() - start_time < timeout:
            try:
                current_size = os.path.getsize(path)
                if current_size == last_size:
                    stable_count += 1
                    if stable_count > 3:  # 300ms stability
                        if self._validate_image(path):
                            return True
                else:
                    last_size = current_size
                    stable_count = 0
                time.sleep(0.1)
            except FileNotFoundError:
                time.sleep(0.1)

        return False

    def _validate_image(self, path):
        """Verify image can be loaded"""
        try:
            with Image.open(path) as img:
                img.verify()
            return True
        except Exception:
            return False


In [3]:
from ipywidgets import widgets
from IPython.display import display as ipython_display
from pyperiscope import Scope
from PIL import Image, ImageDraw, ImageFont
import io
import markdown
import pickle
import dill
import pyautogui
import time
import pyperclip

class Binoculars:
    def __init__(self, width=400):
        self.width = width
        self.image1 = None
        self.image2 = None
        self.box = None
        self.image = None
        self.error_image = self._create_error_image(320,320)
        self._create_widgets()

        self.update_both(self.error_image, self.error_image)
        self.set_description("# No step loaded")

    def load_screenshot_with_data(self, filepath):
        global step
        image = Image.open(filepath)
        
        try:
            # Extract serialized data from metadata
            serialized_data = bytes.fromhex(image.info.get("custom_data", ""))
            data_dict = pickle.loads(serialized_data)
        except (KeyError, pickle.UnpicklingError):
            data_dict = {}  # Return empty dict if no valid data found

        if not data_dict == {}:
            step = Scope(saved_dict=data_dict['step'])
            self.current_step = data_dict['current_step']
            self.last_error = data_dict['last_error']
            self.current_doc = data_dict['current_doc']
            # update UI
            try:
                step.found_locations = data_dict['last_found']
                current = Scope(mouse_offset=step.found_locations[0]['mouse_offset'], area_offset=step.area_offset, area_size=step.area_size, saved_image=image)
                self.update_both(step.render_preview(), current.render_preview())
                self.set_description(self.current_doc)
                self.set_error(self.last_error)
            except:
                self.update_both(step.render_preview(), self.error_image)
                self.set_description(self.current_doc)
                self.set_error(self.last_error)
            

    def _create_error_image(self, width, height):
        # Create black background image
        img = Image.new('RGB', (width, height), color='black')
        draw = ImageDraw.Draw(img)
        
        # Draw gray cross
        gray = (128, 128, 128)
        cross_width = max(2, min(width, height) // 100)  # Dynamic line width
        # Horizontal line
        draw.line((0, height//2, width, height//2), fill=gray, width=cross_width)
        # Vertical line
        draw.line((width//2, 0, width//2, height), fill=gray, width=cross_width)
        
        # Add text
        text = "Image could not be loaded"
        try:
            # Try to load a common font
            font = ImageFont.truetype("arial.ttf", size=min(width, height)//15)
        except:
            # Fallback to default font
            font = ImageFont.load_default()
        
        # Calculate text position
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        x = (width - text_width) // 2
        y = (height - text_height) // 2
        
        # Draw white text with black border for better visibility
        draw.text((x, y), text, fill='white', font=font, stroke_width=2, stroke_fill='black')
        
        return img     
        
    def _create_widgets(self):
        """Initialize all widgets including text areas"""
        # Create description text widget (Markdown)
        self.description = widgets.HTML(
            value='',
            placeholder='Description',
            description='',
        )
        
        # Create image widgets
        self.widget1 = widgets.Image(format='png', width=self.width)
        self.widget2 = widgets.Image(format='png', width=self.width)
        self.image_box = widgets.HBox([self.widget1, self.widget2])
        
        # Create error text widget (Markdown)
        self.error_text = widgets.HTML(
            value='',
            placeholder='Error messages',
            description='',
        )
        
        # Stack all widgets vertically
        self.box = widgets.VBox([
            self.description,
            self.image_box,
            self.error_text
        ])
        
        ipython_display(self.box)
        
    def _convert_to_bytes(self, pil_image):
        """Convert PIL image to bytes"""
        buf = io.BytesIO()
        pil_image.save(buf, format='PNG')
        return buf.getvalue()
    
    def update_both(self, image1, image2):
        """Update both images at once"""
        if not isinstance(image1, Image.Image) or not isinstance(image1, Image.Image):
            raise TypeError("Inputs must be a PIL Image")

        image1_bytes = self._convert_to_bytes(image1)
        image2_bytes = self._convert_to_bytes(image2)
        self.widget1.value = image1_bytes
        self.widget2.value = image2_bytes
    
    def set_description(self, text):
        """Update description text with markdown formatting"""
        # Convert markdown to HTML
        if not text == None:
            md = markdown.Markdown()
            self.description.value = md.convert(text)
    
    def set_error(self, text):
        """Update error text with markdown formatting"""
        # Convert markdown to HTML with red color for errors
        if not text == None:
            md = markdown.Markdown()
            self.error_text.value = md.convert(text)

    def _update_from_screenshot(self, filepath):
        image = Image.open(filepath)
        
        try:
            # Extract serialized data from metadata
            serialized_data = bytes.fromhex(image.info.get("custom_data", ""))
            data_dict = pickle.loads(serialized_data)
        except (KeyError, pickle.UnpicklingError):
            data_dict = {}  # Return empty dict if no valid data found

        # set default values
        left_image = self.error_image
        right_image = self.error_image
        description = str(filepath)[-4:]+"ERROR loading data"
        error_desc =  "could not read the data portion from: "+str(filepath)

        # try to override the values with actual data
        if not data_dict == {}:
            step = Scope(saved_dict=data_dict['step'])
            step.found_locations = data_dict['last_found']
            #self.current_step = data_dict['current_step']
            last_error = data_dict['last_error']
            #self.scope_cell_id = data_dict['scope_cell_id']
            current_doc = data_dict['current_doc']
            current = Scope(mouse_offset=step.found_locations[0]['mouse_offset'], area_offset=step.area_offset, area_size=step.area_size, saved_image=image)
            left_image = step.render_preview()
            right_image = current.render_preview()
            if current_doc:
                description = current_doc
            if last_error:
                error_desc = last_error
            
        # update UI
        self.update_both(left_image, right_image)
        self.set_description(current_doc)
        self.set_error(last_error)
        
    def start_monitoring(self, filepath):
        print("to stop the monitoring use Bincoulars_instance.watcher.stop()")
        if hasattr(self, 'watcher'):
            self.watcher.stop()
        self.watcher = FileWatcher(self, filepath)
        self.watcher.start()

In [4]:
autorunner_pilot = Pilot("odoo-onshape.ipynb", step_timeout=2)

In [ ]:
b.load_screenshot_with_data("_odoo-onshape-8.png")

In [8]:
b.start_monitoring("_odoo-onshape")
autorunner_pilot.run_workbook(firstStep=10)

to stop the monitoring use Bincoulars_instance.watcher.stop()
Staring step: 10
found: 1
Staring step: 11
found: 1
Staring step: 12
found: 1
Staring step: 13
found: 1
Staring step: 14
found: 1
Staring step: 15
found: 1
Staring step: 16
found: 1
Staring step: 17
found: 1
Staring step: 18
found: 1
Staring step: 19
found: 1
found product doc address: https://cad.onshape.com/documents/4eb8d26d28099bb5af424422
Staring step: 20
found: 1
Staring step: 21
found: 1
Staring step: 22
found: 1
Staring step: 23
found: 1
Staring step: 24
found: 1
Staring step: 25
found: 1


In [24]:
b.watcher.stop()

In [25]:
b.start_monitoring("_odoo-onshape")

to stop the monitoring use Bincoulars_instance.watcher.stop()


In [59]:
autorunner_pilot.run_step(0)

Staring step: 0
found: 2


In [78]:
b.start_monitoring("_odoo-onshape")
for i in range(5):
    autorunner_pilot.run_step(i)

to stop the monitoring use Bincoulars_instance.watcher.stop()
Staring step: 0
found: 2
debug: 0
Staring step: 1
Waiting for _odoo-onshape-1.png
try: 0/30
try: 1/30
try: 2/30
found: 1
debug: 1
_odoo-onshape-0.png
deleted
Staring step: 2
found: 1
Waiting for _odoo-onshape-2.png
debug: 2
_odoo-onshape-1.png
deleted
Staring step: 3
found: 1
Waiting for _odoo-onshape-3.png
debug: 3
_odoo-onshape-2.png
deleted
Staring step: 4
found: 1
Waiting for _odoo-onshape-4.png
Found product: Test product 2
debug: 4
_odoo-onshape-3.png
deleted
Waiting for _odoo-onshape-5.png
Waiting for _odoo-onshape-6.png
Waiting for _odoo-onshape-7.png
Waiting for _odoo-onshape-8.png
Waiting for _odoo-onshape-9.png
Waiting for _odoo-onshape-10.png
Waiting for _odoo-onshape-11.png
Waiting for _odoo-onshape-12.png
Waiting for _odoo-onshape-13.png
Waiting for _odoo-onshape-14.png
Waiting for _odoo-onshape-15.png
Waiting for _odoo-onshape-16.png
Waiting for _odoo-onshape-17.png
Waiting for _odoo-onshape-18.png
Waiting for

In [6]:
b = Binoculars(width=320)
#b.start_monitoring("_odoo-onshape.png")